In [3]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

# 1. Carga los datos preprocesados
train = pd.read_csv('Data/train_preprocessed.csv')
test = pd.read_csv('Data/test_preprocessed.csv')

print(f"Forma del dataset de entrenamiento: {train.shape}")
print(f"Distribución de RENDIMIENTO_GLOBAL:")
print(train['RENDIMIENTO_GLOBAL'].value_counts())

# 2. Separa variables predictoras y objetivo
y = train['RENDIMIENTO_GLOBAL']
X = train.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
X_test = test.drop(['ID'], axis=1)

print(f"Número de características: {X.shape[1]}")
print(f"Número de muestras de entrenamiento: {X.shape[0]}")
print(f"Número de muestras de test: {X_test.shape[0]}")

# 3. Pipeline con escalado y SVM
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42, max_iter=2000))  # Incrementamos max_iter
])

# 4. Búsqueda de hiperparámetros específica para clasificación multiclase
param_distributions = [
    # RBF kernel - mejor para problemas no lineales complejos
    {
        'svc__C': [0.1, 1, 10, 100, 1000],
        'svc__kernel': ['rbf'],
        'svc__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
        'svc__class_weight': [None, 'balanced']
    },
    # Linear kernel - bueno para muchas características
    {
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear'],
        'svc__class_weight': [None, 'balanced']
    },
    # Polynomial kernel
    {
        'svc__C': [0.1, 1, 10, 100],
        'svc__kernel': ['poly'],
        'svc__degree': [2, 3],
        'svc__gamma': ['scale', 'auto'],
        'svc__class_weight': [None, 'balanced']
    }
]

# 5. Validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Iniciando búsqueda de hiperparámetros para SVM...")
print("Esto puede tomar varios minutos...")
start_time = time.time()

random_search = RandomizedSearchCV(
    pipe, 
    param_distributions=param_distributions, 
    n_iter=50,
    cv=cv,
    scoring='accuracy', 
    n_jobs=8,  
    verbose=1, 
    random_state=42,
    return_train_score=True
)

random_search.fit(X, y)
elapsed = time.time() - start_time

print(f"\n{'='*50}")
print("RESULTADOS DE LA BÚSQUEDA DE HIPERPARÁMETROS")
print(f"{'='*50}")
print(f"Mejores parámetros: {random_search.best_params_}")
print(f"Mejor score de validación cruzada: {random_search.best_score_:.4f}")
print(f"Tiempo de búsqueda: {elapsed/60:.2f} minutos")

# 6. Análisis del modelo seleccionado
best_model = random_search.best_estimator_
print(f"\nModelo seleccionado:")
print(f"- Kernel: {random_search.best_params_['svc__kernel']}")
print(f"- C: {random_search.best_params_['svc__C']}")
if 'svc__gamma' in random_search.best_params_:
    print(f"- Gamma: {random_search.best_params_['svc__gamma']}")
if 'svc__degree' in random_search.best_params_:
    print(f"- Degree: {random_search.best_params_['svc__degree']}")
print(f"- Class weight: {random_search.best_params_['svc__class_weight']}")

# 7. Predicciones en el conjunto de test
print("\nGenerando predicciones para el conjunto de test...")
y_pred_test = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# 8. Validación cruzada detallada
print("\nCalculando métricas de validación cruzada...")
y_pred_cv = cross_val_predict(best_model, X, y, cv=cv)
accuracy_cv = accuracy_score(y, y_pred_cv)

print(f"Accuracy en validación cruzada: {accuracy_cv:.4f}")

# 9. Visualizaciones
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Matriz de confusión
cm = confusion_matrix(y, y_pred_cv)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=sorted(y.unique()), yticklabels=sorted(y.unique()))
axes[0].set_xlabel('Predicho')
axes[0].set_ylabel('Real')
axes[0].set_title(f'Matriz de Confusión\n(Accuracy: {accuracy_cv:.4f})')

# Distribución de predicciones en test
pred_counts = pd.Series(y_pred_test).value_counts().sort_index()
axes[1].bar(pred_counts.index, pred_counts.values, alpha=0.7)
axes[1].set_xlabel('Rendimiento Global')
axes[1].set_ylabel('Número de predicciones')
axes[1].set_title('Distribución de predicciones en Test')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 10. Reporte detallado
print(f"\n{'='*50}")
print("REPORTE DE CLASIFICACIÓN")
print(f"{'='*50}")
print(classification_report(y, y_pred_cv))

# 11. Guardar resultados
submission = pd.DataFrame({
    'ID': test['ID'], 
    'RENDIMIENTO_GLOBAL': y_pred_test
})
submission.to_csv('submission_svm_optimizado.csv', index=False)

# Guardar también las probabilidades
prob_df = pd.DataFrame(y_pred_proba, columns=best_model.classes_)
prob_df['ID'] = test['ID']
prob_df['RENDIMIENTO_GLOBAL'] = y_pred_test
prob_df.to_csv('submission_svm_probabilidades.csv', index=False)

print(f"\nArchivos generados:")
print(f"- submission_svm_optimizado.csv (predicciones)")
print(f"- submission_svm_probabilidades.csv (con probabilidades)")

# 12. Estadísticas adicionales
print(f"\n{'='*50}")
print("ESTADÍSTICAS ADICIONALES")
print(f"{'='*50}")
print(f"Número de vectores de soporte: {best_model.named_steps['svc'].n_support_}")
print(f"Distribución original:")
for clase in sorted(y.unique()):
    count = (y == clase).sum()
    pct = count / len(y) * 100
    print(f"  {clase}: {count} ({pct:.1f}%)")

print(f"\nDistribución predicha en test:")
for clase in sorted(pred_counts.index):
    count = pred_counts[clase]
    pct = count / len(y_pred_test) * 100
    print(f"  {clase}: {count} ({pct:.1f}%)")

Forma del dataset de entrenamiento: (578824, 21)
Distribución de RENDIMIENTO_GLOBAL:
RENDIMIENTO_GLOBAL
medio-bajo    151046
bajo          149170
medio-alto    146099
alto          132509
Name: count, dtype: int64
Número de características: 19
Número de muestras de entrenamiento: 578824
Número de muestras de test: 250829
Iniciando búsqueda de hiperparámetros para SVM...
Esto puede tomar varios minutos...
Fitting 5 folds for each of 50 candidates, totalling 250 fits


KeyboardInterrupt: 